In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import gridspec
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pathlib
import PIL
import PIL.Image
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler 

# Data Preprocessing

In [ ]:
img_path = '../input/plant-seedlings-classification/train'
img_path = pathlib.Path(img_path)
image_count = len(list(img_path.glob('*/*.png')))
print(image_count)

In [ ]:
plant = list(img_path.glob('Sugar beet/*'))
print(len(plant))
PIL.Image.open(str(plant[1]))

In [ ]:
batch_size = 32
img_size = (299,299)

In [ ]:
train_ds = image_dataset_from_directory(
  img_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_size),
  batch_size=batch_size)

In [ ]:
val_ds = image_dataset_from_directory(
  img_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_size),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
train_ds_per_class = {}
for cls_name in class_names:
    train_ds_per_class[cls_name] = len(list(img_path.glob(cls_name+'/*')))
print(train_ds_per_class)

In [ ]:
train_class = list(train_ds_per_class.keys())
values = list(train_ds_per_class.values())
fig = plt.figure(figsize = (15,5))

plt.bar(train_class, values, width = 0.6)
plt.show()

In [ ]:
test_img_path = '../input/plant-seedlings-classification/test'
test_img_path = pathlib.Path(test_img_path)
print(test_img_path)

new_test_path = pathlib.Path('/kaggle/working/test/images')

In [ ]:
import shutil
shutil.copytree(str(test_img_path), str(new_test_path))

In [ ]:
import os
def parse_image(filename):
    parts = tf.strings.split(filename, os.path.sep)
    label = parts[-1]

    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image)
#     image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, img_size)
    return image, label

In [ ]:
test_ds = tf.data.Dataset.list_files(str(new_test_path)+'/*.png') \
    .map(parse_image) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

In [ ]:
for image_batch, label_batch in test_ds.take(1):
    print(image_batch.numpy().min())
    print(image_batch.numpy().max())
#     show(image_batch[0],label_batch[0])
    print(image_batch[0])

In [ ]:
def show(image, label):
    plt.figure()
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(label.numpy().decode('utf-8'))
    plt.axis('off')
    
    
for image_batch, label_batch in test_ds.take(1):
    print(image_batch.numpy().mean())
    show(image_batch[0],label_batch[0])
    print(image_batch[0])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for image_batch, _ in train_ds.take(1):
    first_image = image_batch[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0].numpy().astype('uint8'))
        plt.axis('off')

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

# Defining Model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.trainable = False
# base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(12,activation="softmax")
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
inputs = tf.keras.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)

x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(128,activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(12)(x)
# outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

# Model Training

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
loss0, accuracy0 = model.evaluate(val_ds)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
EarlyStop_callback = EarlyStopping(min_delta=0.001, patience=10, restore_best_weights=True)

In [ ]:
EPOCHS = 30
history = model.fit(train_ds,
                    epochs=EPOCHS,
                    validation_data=val_ds,
                   callbacks = [EarlyStop_callback])

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
history_df.loc[:, ['accuracy', 'val_accuracy']].plot();
print("Maximum validation accuracy: {}".format(history_df['val_accuracy'].max()))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.xticks(list(range(20)))
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xticks(list(range(20)))
plt.xlabel('Epoch')
plt.show()

# Fine Tuning

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
fine_tune_from = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_from]:
    layer.trainable =  False

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])
model.summary()

In [ ]:
fine_tune_epochs = 5
total_epochs = EPOCHS + fine_tune_epochs

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# Prediction

In [ ]:
for image_batch, label_batch in test_ds.shuffle(400).take(1):
    predictions = model.predict_on_batch(image_batch)
    predictions = tf.nn.softmax(predictions)
    predictions = predictions.numpy()
    
    class_indices = np.argmax(predictions, axis = 1)
    predicted_class = []
#     print(label_batch)
    for index in class_indices:
        predicted_class.append(str(class_names[index]))
    print(class_indices)
    print(predicted_class)
#     print(predictions)

In [ ]:
predictions = []
image_ids = []
for image_batch, label_batch in test_ds:
    batch_predictions = model.predict_on_batch(image_batch)
    batch_predictions = tf.nn.softmax(batch_predictions)
    batch_predictions = batch_predictions.numpy()
    
    class_indices = np.argmax(batch_predictions, axis = 1)
    predicted_class = []
    for index in class_indices:
        predicted_class.append(str(class_names[index]))
    predictions += predicted_class
    
    str_img_ids = []
    for img_id in label_batch:
        str_img_ids.append(img_id.numpy().decode("utf-8"))
    
#     image_ids += label_batch.numpy().tolist()
    image_ids += str_img_ids

In [ ]:
submission = pd.DataFrame({'file': image_ids, 'species': predictions})
submission.head()

In [ ]:
submission.to_csv('submission_plant.csv',index=False)

In [ ]:
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]
class_list = []

for i in range(0, predictions.shape[0]):
    y_class = predictions[i, :].argmax(axis=-1)
    class_list += [species_list[y_class]]

submission_1 = pd.DataFrame()
submission_1['file'] = sample_submission['file']
submission_1['species'] = class_list

submission_1.to_csv('submission_1.csv', index=False)

print('Submission file generated. All done.')

In [ ]:
submission_1.head()

In [ ]:
sample_submission = pd.read_csv("../input/plant-seedlings-classification/sample_submission.csv")

In [ ]:
sample_submission = pd.read_csv("../input/plant-seedlings-classification/sample_submission.csv")
submission.head()

In [ ]:
submission.loc[:,class_names] = predictions